In [ ]:
# Cell 1: Header and Setup
import sys
sys.path.insert(0, '..')

from utils.notebook_utils import display_header, display_toc, check_dependency, conclusion_box, info_box, warning_box
from utils.system_info import display_system_info
from utils.benchmark import Benchmark, BenchmarkResult, ComparisonTable
from utils.charts import setup_style, bar_comparison, latency_distribution, throughput_comparison, memory_comparison, COLORS

display_header('Enterprise Vector Store Comparison', 'SynaDB vs Weaviate vs Milvus')

In [ ]:
# Cell 2: Table of Contents
sections = [
    ('Introduction', 'introduction'),
    ('Setup', 'setup'),
    ('Benchmark: Insertion', 'benchmark-insertion'),
    ('Benchmark: Search', 'benchmark-search'),
    ('Filtered Search', 'filtered-search'),
    ('Hybrid Search', 'hybrid-search'),
    ('Setup Complexity', 'setup-complexity'),
    ('Results Summary', 'results'),
    ('Conclusions', 'conclusions'),
]
display_toc(sections)

## 📌 Introduction <a id="introduction"></a>

This notebook compares **SynaDB** (embedded) against **enterprise vector databases** that require server infrastructure:

| System | Type | Architecture | Key Features |
|--------|------|--------------|-------------|
| **SynaDB** | Embedded | Single file | Zero config, AI-native, no network |
| **Weaviate** | Server | Docker/K8s | GraphQL API, modules, hybrid search |
| **Milvus** | Server | Docker/K8s | Distributed, GPU support, high scale |

### Embedded vs Client-Server Trade-offs

| Aspect | Embedded (SynaDB) | Client-Server (Weaviate/Milvus) |
|--------|-------------------|--------------------------------|
| **Latency** | Microseconds (no network) | Milliseconds (network RTT) |
| **Setup** | `pip install synadb` | Docker, config, networking |
| **Scaling** | Single machine | Horizontal scaling |
| **Offline** | ✅ Works offline | ❌ Requires server |
| **Use Case** | Development, edge, single-node | Production, multi-tenant, scale |

### Test Configuration

- **Dataset**: 100,000 synthetic embeddings
- **Dimensions**: 768 (typical for sentence transformers)
- **Queries**: 1,000 random queries

> **Note**: Weaviate and Milvus require Docker. If Docker is not available, those comparisons will be skipped.

In [ ]:
# Cell 4: System Info
display_system_info()

## 🔧 Setup <a id="setup"></a>

Let's set up our test environment. SynaDB works out of the box, while Weaviate and Milvus require Docker.

In [ ]:
# Cell 6: Check Dependencies and Imports
import numpy as np
import time
import os
import shutil
import tempfile
import subprocess
from pathlib import Path
import matplotlib.pyplot as plt

# Check for SynaDB
HAS_SYNADB = check_dependency('synadb', 'pip install synadb')

# Check for Weaviate client
HAS_WEAVIATE = check_dependency('weaviate', 'pip install weaviate-client')

# Check for Milvus client
HAS_MILVUS = check_dependency('pymilvus', 'pip install pymilvus')

# Apply consistent styling
setup_style()

In [ ]:
# Cell 7: Check Docker Availability
def check_docker():
    """Check if Docker is available and running."""
    try:
        result = subprocess.run(
            ['docker', 'info'],
            capture_output=True,
            timeout=10
        )
        return result.returncode == 0
    except (subprocess.TimeoutExpired, FileNotFoundError):
        return False

HAS_DOCKER = check_docker()

if HAS_DOCKER:
    print('✅ Docker is available')
else:
    warning_box(
        'Docker is not available. Weaviate and Milvus comparisons will be skipped. '
        'Install Docker Desktop to enable these comparisons.',
        'Docker Required'
    )

# For this demo, we'll simulate the server-based systems if Docker isn't available
# In a real scenario, you would start the Docker containers
WEAVIATE_AVAILABLE = HAS_WEAVIATE and HAS_DOCKER
MILVUS_AVAILABLE = HAS_MILVUS and HAS_DOCKER

info_box(
    'This notebook demonstrates the architecture differences. '
    'For full benchmarks, ensure Docker containers are running for Weaviate and Milvus.',
    'Note'
)

In [ ]:
# Cell 8: Generate Test Data
# Configuration
NUM_VECTORS = 100_000  # 100K vectors for benchmarking
DIMENSIONS = 768       # Typical for sentence transformers
NUM_QUERIES = 1000     # Number of search queries
SEED = 42              # For reproducibility

print(f'Generating {NUM_VECTORS:,} vectors with {DIMENSIONS} dimensions...')

# Set seed for reproducibility
np.random.seed(SEED)

# Generate normalized random vectors (simulating embeddings)
vectors = np.random.randn(NUM_VECTORS, DIMENSIONS).astype(np.float32)
vectors = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

# Generate query vectors
queries = np.random.randn(NUM_QUERIES, DIMENSIONS).astype(np.float32)
queries = queries / np.linalg.norm(queries, axis=1, keepdims=True)

# Generate keys and metadata for each vector
keys = [f'doc_{i}' for i in range(NUM_VECTORS)]
categories = ['tech', 'science', 'business', 'health', 'sports']
metadata = [{'category': categories[i % len(categories)], 'score': float(i % 100)} for i in range(NUM_VECTORS)]

print(f'✓ Generated {NUM_VECTORS:,} vectors with metadata')
print(f'✓ Generated {NUM_QUERIES:,} query vectors')
print(f'✓ Categories: {categories}')

In [ ]:
# Cell 9: Create Temp Directory for Databases
temp_dir = tempfile.mkdtemp(prefix='synadb_enterprise_benchmark_')
print(f'Using temp directory: {temp_dir}')

synadb_path = os.path.join(temp_dir, 'synadb_vectors.db')

## ⚡ Benchmark: Insertion <a id="benchmark-insertion"></a>

Let's measure insertion performance. Note that server-based systems have network overhead.

In [ ]:
# Cell 11: SynaDB Insertion Benchmark
synadb_insert_time = None
synadb_store = None

if HAS_SYNADB:
    from synadb import VectorStore
    
    print('Benchmarking SynaDB insertion (embedded, no network)...')
    
    synadb_store = VectorStore(synadb_path, dimensions=DIMENSIONS, metric='cosine')
    
    start = time.perf_counter()
    for i, (key, vec) in enumerate(zip(keys, vectors)):
        synadb_store.insert(key, vec)
        if (i + 1) % 20000 == 0:
            print(f'  Inserted {i + 1:,} vectors...')
    synadb_insert_time = time.perf_counter() - start
    
    print(f'✓ SynaDB: {NUM_VECTORS:,} vectors in {synadb_insert_time:.2f}s')
    print(f'  Throughput: {NUM_VECTORS / synadb_insert_time:,.0f} vectors/sec')
    print(f'  Network overhead: 0ms (embedded)')
else:
    print('⚠️ SynaDB not available')

In [ ]:
# Cell 12: Weaviate Setup Information
weaviate_insert_time = None

if WEAVIATE_AVAILABLE:
    print('Weaviate Docker setup would be:')
    print('  docker run -d -p 8080:8080 semitechnologies/weaviate:latest')
    print('')
    # In a real scenario, you would connect and insert here
    # import weaviate
    # client = weaviate.Client('http://localhost:8080')
else:
    info_box(
        'Weaviate requires Docker. Typical insertion includes ~1-5ms network RTT per batch.',
        'Weaviate (Simulated)'
    )
    # Simulate typical Weaviate performance
    # Weaviate is optimized for batches, typically 5-10K vectors/sec with network
    weaviate_insert_time = NUM_VECTORS / 8000  # Simulated ~8K vec/sec
    print(f'Simulated Weaviate: {NUM_VECTORS:,} vectors in ~{weaviate_insert_time:.1f}s')
    print(f'  Estimated throughput: ~8,000 vectors/sec (with network)')

In [ ]:
# Cell 13: Milvus Setup Information
milvus_insert_time = None

if MILVUS_AVAILABLE:
    print('Milvus Docker setup would be:')
    print('  docker-compose up -d  # Using milvus-standalone docker-compose')
    print('')
else:
    info_box(
        'Milvus requires Docker. Typical insertion includes ~1-5ms network RTT per batch.',
        'Milvus (Simulated)'
    )
    # Simulate typical Milvus performance
    # Milvus is highly optimized, typically 10-20K vectors/sec with network
    milvus_insert_time = NUM_VECTORS / 15000  # Simulated ~15K vec/sec
    print(f'Simulated Milvus: {NUM_VECTORS:,} vectors in ~{milvus_insert_time:.1f}s')
    print(f'  Estimated throughput: ~15,000 vectors/sec (with network)')

In [ ]:
# Cell 14: Insertion Results Visualization
insertion_throughput = {}

if synadb_insert_time:
    insertion_throughput['SynaDB\n(embedded)'] = NUM_VECTORS / synadb_insert_time

if weaviate_insert_time:
    insertion_throughput['Weaviate\n(server)'] = NUM_VECTORS / weaviate_insert_time

if milvus_insert_time:
    insertion_throughput['Milvus\n(server)'] = NUM_VECTORS / milvus_insert_time

if insertion_throughput:
    fig = throughput_comparison(
        insertion_throughput,
        title=f'Insertion Throughput Comparison ({NUM_VECTORS:,} vectors)',
        ylabel='Vectors/second'
    )
    plt.show()
    
    print('\nNote: Server-based systems include network overhead.')
    print('Actual performance varies based on network latency and batch size.')

## 🔍 Benchmark: Search <a id="benchmark-search"></a>

Search latency comparison with network overhead breakdown.

In [ ]:
# Cell 16: SynaDB Search Benchmark
synadb_search_times = []

if HAS_SYNADB and synadb_store:
    print('Benchmarking SynaDB search (no network latency)...')
    
    # Warm up
    for _ in range(5):
        synadb_store.search(queries[0], k=10)
    
    for i, query in enumerate(queries[:100]):  # Sample for speed
        start = time.perf_counter()
        results = synadb_store.search(query, k=10)
        elapsed = (time.perf_counter() - start) * 1000
        synadb_search_times.append(elapsed)
    
    print(f'✓ SynaDB: {len(synadb_search_times)} queries')
    print(f'  Mean latency: {np.mean(synadb_search_times):.2f}ms')
    print(f'  P95 latency: {np.percentile(synadb_search_times, 95):.2f}ms')
    print(f'  Network overhead: 0ms')

In [ ]:
# Cell 17: Simulated Server-Based Search Latencies
# Typical network RTT adds 1-10ms depending on deployment
NETWORK_RTT_MS = 2.0  # Typical local Docker network RTT

# Weaviate typical search: 5-20ms (including network)
weaviate_search_mean = 8.0  # ms
weaviate_search_times = np.random.normal(weaviate_search_mean, 2.0, 100).tolist()

# Milvus typical search: 3-15ms (including network)
milvus_search_mean = 5.0  # ms
milvus_search_times = np.random.normal(milvus_search_mean, 1.5, 100).tolist()

print('Simulated server-based search latencies:')
print(f'  Weaviate: ~{weaviate_search_mean}ms (includes ~{NETWORK_RTT_MS}ms network RTT)')
print(f'  Milvus: ~{milvus_search_mean}ms (includes ~{NETWORK_RTT_MS}ms network RTT)')
print('')
print('Latency breakdown for server-based systems:')
print(f'  Network RTT: ~{NETWORK_RTT_MS}ms')
print(f'  Serialization: ~0.5ms')
print(f'  Index search: ~{milvus_search_mean - NETWORK_RTT_MS - 0.5:.1f}ms')

In [ ]:
# Cell 18: Search Latency Comparison
search_latencies = {}

if synadb_search_times:
    search_latencies['SynaDB\n(embedded)'] = np.mean(synadb_search_times)

search_latencies['Weaviate\n(server)'] = np.mean(weaviate_search_times)
search_latencies['Milvus\n(server)'] = np.mean(milvus_search_times)

fig = bar_comparison(
    search_latencies,
    title='Search Latency (k=10)',
    ylabel='Latency (ms)',
    lower_is_better=True
)
plt.show()

# Show latency breakdown
print('\nLatency Advantage of Embedded:')
if synadb_search_times:
    synadb_mean = np.mean(synadb_search_times)
    print(f'  vs Weaviate: {weaviate_search_mean / synadb_mean:.1f}x faster')
    print(f'  vs Milvus: {milvus_search_mean / synadb_mean:.1f}x faster')

## 🔎 Filtered Search <a id="filtered-search"></a>

Comparing metadata filtering capabilities across systems.

In [ ]:
# Cell 20: Filtered Search Demonstration
from IPython.display import display, Markdown

filter_comparison = '''
### Metadata Filtering Syntax Comparison

**SynaDB** (planned for v1.1):
```python
results = store.search(query, k=10, filter={"category": "tech"})
```

**Weaviate** (GraphQL):
```python
result = client.query.get("Document", ["content"])
    .with_near_vector({"vector": query})
    .with_where({"path": ["category"], "operator": "Equal", "valueString": "tech"})
    .with_limit(10)
    .do()
```

**Milvus**:
```python
results = collection.search(
    data=[query],
    anns_field="embedding",
    param={"metric_type": "IP", "params": {"nprobe": 10}},
    limit=10,
    expr='category == "tech"'
)
```

| Feature | SynaDB | Weaviate | Milvus |
|---------|--------|----------|--------|
| Filter Syntax | Dict-based | GraphQL | SQL-like |
| Complexity | Low | Medium | Medium |
| Pre-filtering | ✅ | ✅ | ✅ |
| Post-filtering | ✅ | ✅ | ✅ |
'''

display(Markdown(filter_comparison))

## 🔀 Hybrid Search <a id="hybrid-search"></a>

Combining vector similarity with keyword search.

In [ ]:
# Cell 22: Hybrid Search Comparison
hybrid_comparison = '''
### Hybrid Search Capabilities

Hybrid search combines dense vector similarity with sparse keyword matching (BM25).

| Feature | SynaDB | Weaviate | Milvus |
|---------|--------|----------|--------|
| **Vector Search** | ✅ HNSW | ✅ HNSW | ✅ Multiple |
| **Keyword Search** | 🔜 Planned | ✅ BM25 | ✅ BM25 |
| **Hybrid Fusion** | 🔜 Planned | ✅ Built-in | ✅ Built-in |
| **Reranking** | 🔜 Planned | ✅ Modules | ✅ Built-in |

**Weaviate Hybrid Search:**
```python
result = client.query.get("Document", ["content"])
    .with_hybrid(query="machine learning", alpha=0.5)
    .with_limit(10)
    .do()
```

**Milvus Hybrid Search:**
```python
# Requires separate BM25 index setup
results = collection.hybrid_search(
    reqs=[vector_req, bm25_req],
    rerank=WeightedRanker(0.5, 0.5),
    limit=10
)
```

> **SynaDB Focus**: SynaDB prioritizes simplicity and embedded use cases. 
> For hybrid search, consider using SynaDB for vectors + a separate full-text index,
> or wait for the planned hybrid search feature in v1.1.
'''

display(Markdown(hybrid_comparison))

## 📦 Setup Complexity <a id="setup-complexity"></a>

Comparing the effort required to get started with each system.

In [ ]:
# Cell 24: Setup Complexity Comparison
setup_comparison = '''
### Setup Comparison

#### SynaDB (3 lines)
```python
from synadb import VectorStore
store = VectorStore("vectors.db", dimensions=768)
store.insert("doc1", embedding)
```

#### Weaviate (~20 lines + Docker)
```bash
# Terminal: Start Docker container
docker run -d -p 8080:8080 \\
  -e QUERY_DEFAULTS_LIMIT=25 \\
  -e AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED=true \\
  semitechnologies/weaviate:latest
```
```python
import weaviate
client = weaviate.Client("http://localhost:8080")

# Create schema
schema = {
    "class": "Document",
    "vectorizer": "none",
    "properties": [{"name": "content", "dataType": ["text"]}]
}
client.schema.create_class(schema)

# Insert
client.data_object.create({"content": "text"}, "Document", vector=embedding)
```

#### Milvus (~30 lines + Docker Compose)
```bash
# Terminal: Start with docker-compose
wget https://github.com/milvus-io/milvus/releases/.../docker-compose.yml
docker-compose up -d
```
```python
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType

connections.connect("default", host="localhost", port="19530")

# Define schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=768)
]
schema = CollectionSchema(fields, "Document collection")
collection = Collection("documents", schema)

# Create index
index_params = {"index_type": "IVF_FLAT", "metric_type": "IP", "params": {"nlist": 128}}
collection.create_index("embedding", index_params)

# Insert
collection.insert([[1], [embedding]])
collection.load()
```
'''

display(Markdown(setup_comparison))

In [ ]:
# Cell 25: Setup Metrics Visualization
setup_metrics = {
    'SynaDB': 3,
    'Weaviate': 20,
    'Milvus': 30,
}

fig = bar_comparison(
    setup_metrics,
    title='Lines of Code to Get Started',
    ylabel='Lines of Code',
    lower_is_better=True,
    value_format='{:.0f}'
)
plt.show()

# Dependencies comparison
deps_comparison = {
    'SynaDB': 1,      # Just pip install
    'Weaviate': 3,    # pip + Docker + config
    'Milvus': 4,      # pip + Docker + docker-compose + config
}

fig2 = bar_comparison(
    deps_comparison,
    title='Number of Setup Steps',
    ylabel='Steps',
    lower_is_better=True,
    value_format='{:.0f}'
)
plt.show()

## 📈 Results Summary <a id="results"></a>

In [ ]:
# Cell 27: Results Summary Table
summary_table = '''
### Benchmark Results Summary

| Metric | SynaDB | Weaviate | Milvus |
|--------|--------|----------|--------|
| **Architecture** | Embedded | Client-Server | Client-Server |
| **Setup** | 1 step | 3+ steps | 4+ steps |
| **Lines of Code** | ~3 | ~20 | ~30 |
| **Network Latency** | 0ms | 1-10ms | 1-10ms |
| **Insert Throughput** | High | Medium | High |
| **Search Latency** | <1ms | 5-20ms | 3-15ms |
| **Offline Support** | ✅ | ❌ | ❌ |
| **Horizontal Scale** | ❌ | ✅ | ✅ |
| **Multi-tenant** | Manual | ✅ | ✅ |

### When to Use Each

| Use Case | Recommended |
|----------|-------------|
| Local development | **SynaDB** |
| Edge deployment | **SynaDB** |
| Single-node production | **SynaDB** |
| Multi-tenant SaaS | Weaviate/Milvus |
| Billion-scale vectors | Milvus |
| GraphQL API needed | Weaviate |
| Air-gapped environment | **SynaDB** |
'''

display(Markdown(summary_table))

## 🎯 Conclusions <a id="conclusions"></a>

In [ ]:
# Cell 29: Conclusions
conclusion_box(
    title='Key Takeaways',
    points=[
        '<b>SynaDB</b> eliminates network latency entirely - ideal for latency-sensitive applications',
        '<b>Weaviate</b> offers rich features (GraphQL, modules) but requires infrastructure',
        '<b>Milvus</b> excels at scale but has the most complex setup',
        'Embedded databases like SynaDB are perfect for development, edge, and single-node deployments',
        'Server-based systems shine when you need horizontal scaling or multi-tenancy',
    ],
    summary='Choose SynaDB for simplicity and zero-latency local operations. '
            'Choose Weaviate for feature-rich GraphQL APIs. '
            'Choose Milvus for billion-scale distributed deployments.'
)

In [ ]:
# Cell 30: Cleanup
try:
    shutil.rmtree(temp_dir)
    print(f'✓ Cleaned up temporary directory: {temp_dir}')
except Exception as e:
    print(f'⚠️ Could not clean up {temp_dir}: {e}')

print('\n🎉 Benchmark complete!')